In [1]:
from eumap.misc import find_files, nan_percentile
from eumap.raster import read_rasters, save_rasters, write_new_raster
from eumap.parallel import TilingProcessing

import time as tm
import numpy as np
import os
# import ee
# import geemap
import multiprocessing
from eumap.raster import read_rasters
import rasterio
from rasterio import features
import geopandas as gpd
from osgeo import ogr, gdal
from multiprocessing import Pool
import warnings
from os.path import exists
import pandas as pd
from rasterio.features import rasterize

# Suppress all warnings
warnings.filterwarnings("ignore")

In [12]:
li = ['RU', 'GL0000', 'AZ', 'BY', 'GE', 'PM', 'AM', 'SX', 'BL97700', 'MF', None]
poly1 = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/PolbndA_optionRS.shp')
poly2 = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/PolbndA.shp')
ineu = [i for i in poly2['SHN0'].unique() if i not in li]
poly3 = poly2.loc[poly2['SHN0'].isin(ineu)]
geo = pd.concat([poly1, poly3])
geo.dropna(subset=['SHN0'], inplace=True)
geo['code'] = geo['SHN0'].str[0:2]

code = pd.read_csv('/mnt/inca/EU_landmask/iso_3366_1_country_code.csv')
# code.loc[code['alpha-2']=='GR','alpha-2']='EL'
# code.loc[code['alpha-2']=='GB','alpha-2']='UK'
geo['indexx'] = 0
for i in geo['code'].unique().tolist():
    temp = code.loc[code['alpha-2']==i]['country-code']
    if len(temp)==0:
        continue
    geo.loc[geo['code']==i,'indexx'] = temp.values[0]
    
input_files = find_files('/mnt/inca/EU_landmask/landmask3','*.tif')
input_files = [str(i) for i in input_files]
shapes = [(geometry, indexx) for geometry, indexx in zip(geo.geometry, geo.indexx)]
files = [(shapes,i) for i in input_files]
# files = [(geo,code,i) for i in input_files]

In [15]:
def worker_iso(shapes,ref):
#     start = tm.time()
    mask_ds = rasterio.open(ref)
    mask = np.zeros(mask_ds.shape)
    refim = mask_ds.read()
    refim = np.squeeze(refim)
    out_f = '/mnt/inca/EU_landmask/landmask_euroworld2/iso_EU' + ref[43::]
    
    rasterize(shapes, out=mask, transform=mask_ds.transform)
    mask[refim==3]=0
    save_rasters(ref, [out_f], mask, dtype='float32' ,n_jobs=30)
#     end = tm.time()
#     print(f'{round((end-start)/60)} mins used')
    return

In [16]:
# test = ['/mnt/inca/EU_landmask/landmask3/landmask_EU_10m_2021_v200_N45E012_Map.tif']
# #        '/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N51E006_Map.tif',
# #        '/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N48E000_Map.tif']

# for i in test:
#     result = worker_iso(shapes,i)
    
with Pool(20) as p:
    result = p.starmap(worker_iso, files)

In [5]:
def worker_country(geo,code,f):
    mask_ds = rasterio.open(f)
    out_f = '/mnt/inca/EU_landmask/landmask_euroworld2/countrymap_EU' + f[43::]
#     nut = nut.to_crs(mask_ds.crs)
#     if exists(out_f):
#         return None
    
    # Create mask
    mask = np.zeros(mask_ds.shape)
    mask_im = mask_ds.read()
    mask_im = np.squeeze(mask_im)
    mask[mask_im==3.0] = 3
    
    for cntr in geo['SHN0'].unique().tolist():
        cnt = cntr[0:2]
        temp = code.loc[code['alpha-2']==cnt]['country-code']
        if len(temp)==0:
            continue
        value = temp.values[0]
        inut = geo.loc[geo['SHN0']==cntr]
        isnut = features.geometry_mask(inut['geometry'].tolist(), out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
        isnut = ~isnut
        if np.any(isnut):
            mask[isnut] = value
        
#     if ~np.any(is_inland):
#         a = f.split('_')[-2]
#         print(f'{a} out of scope')
#         return a
#     out_t = '/mnt/inca/EU_landmask/landmask/test_inland' + f[55::]
#     save_rasters(f, [out_t], is_inland, n_jobs=30)
#     print(np.unique(mask))
    save_rasters(f, [out_f], mask, dtype='float32' ,n_jobs=30)
    return None

In [6]:
test = ['/mnt/inca/EU_landmask/landmask3/landmask_EU_10m_2021_v200_N45E012_Map.tif']
#        '/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N51E006_Map.tif',
#        '/mnt/inca/EU_landmask/landmask2/landmask_EU_10m_2021_v200_N48E000_Map.tif']

for i in test:
    result = worker_country(geo,code,i)

# with Pool(12) as p:
#     result = p.starmap(worker_country, files)

In [9]:
vrt_file = '/mnt/inca/EU_landmask/landmask_euroworld/euroworld_landmask.vrt'
# input_files = find_files('/mnt/inca/EU_landmask/landmask_euroworld/','countrymap*.tif')
# input_files = [str(i) for i in input_files]
# vrt_dataset = gdal.BuildVRT(vrt_file, input_files)
# vrt_dataset.FlushCache()

In [8]:
tif_file = '/mnt/inca/EU_landmask/euroworld_landmask.tif'
ref_file = '/mnt/inca/EU_landmask/U2018_CHA1218_12_V2020_20u1.tif'
with rasterio.open(ref_file) as src:
    projection = src.crs
    transform = src.bounds

# gdal.SetConfigOption('CHECK_DISK_FREE_SPACE', 'FALSE')
# gdal.Warp(tif_file, vrt_file, format='GTiff', outputBounds=transform, dstSRS=projection, xRes=10, yRes=-10)

In [17]:
input_files = find_files('/mnt/inca/EU_landmask/landmask_euroworld2','*.tif')
input_files = [str(i) for i in input_files]
output_vrt = '/mnt/inca/EU_landmask/landmask_euroworld2/iso_landmask.vrt'
ds = gdal.BuildVRT(output_vrt, input_files)
ds.FlushCache()

In [9]:
src.bounds

BoundingBox(left=900000.0, bottom=900000.0, right=7400000.0, top=5500000.0)